<div style="width:100%;text-align: center;">
    <img src="https://user.oc-static.com/upload/2019/02/24/15510245026714_Seattle_logo_landscape_blue-black.png" />
</div>

# Introduction

Vous travaillez pour la ville de Seattle. Pour atteindre son objectif de ville neutre en émissions de carbone en 2050, votre équipe s’intéresse de près aux émissions des bâtiments non destinés à l’habitation.

Des relevés minutieux ont été effectués par vos agents en 2015 et en 2016. Cependant, ces relevés sont coûteux à obtenir, et à partir de ceux déjà réalisés, vous voulez tenter de prédire les émissions de CO2 et la consommation totale d’énergie de bâtiments pour lesquels elles n’ont pas encore été mesurées.

Votre prédiction se basera sur les données déclaratives du permis d'exploitation commerciale (taille et usage des bâtiments, mention de travaux récents, date de construction..)

Vous cherchez également à évaluer l’intérêt de l’"ENERGY STAR Score" pour la prédiction d’émissions, qui est fastidieux à calculer avec l’approche utilisée actuellement par votre équipe.


# Sommaire

1. [Prétraitement](#preprcessing)  
    1.1. [Chargement et contrôle des données](#data-load-and-check)  
    1.2. [Préparation des jeux de données](#data-prepare)   
    1.2. [Cas des outliers](#outliers)
2. [Régression linéaire basique](#simple-linear-regression)  
3. [Comparaison des modèles](#model-compare)  
4. [Comparaison des modèles sans le EnergyStarScore](#model-compare-without-ESS)  
5. [Conclusion](#conclude)

In [ ]:
# import des bibliothèques utilisées
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge, SGDRegressor, ElasticNet
from sklearn.svm import SVR
from sklearn.preprocessing import *
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.metrics import *
from sklearn.ensemble import RandomForestRegressor

from xgboost import XGBRegressor

# <a id="preprocessing">Prétraitement<a/> 

##  <a id="data-load-and-check">Chargement et contrôle des données<a/>   

In [ ]:
data = pd.read_csv("/kaggle/input/refined-data/data_hard_refined.csv")
print(data.columns)
print(data.shape)
data.head()

In [ ]:
data.describe()

In [ ]:
print(data.ComplianceStatus.unique())
print(data.DefaultData.unique())

##  <a id="data-prepare">Préparation des jeux de données<a/>   

Pour répondre au consigne de l'énoncé nous devons supprimer les données liées à la consommation énergétique car les relevés sont coûteux à obtenir. Nous supprimons donc la colonne 'SteamUse(kBtu)'

In [ ]:
data = data.drop(['SteamUse(kBtu)'], axis=1)

In [ ]:
data.info()

Certaines colonnes ne sont pas numérique, il va falloir les modifier. Analysons plus en détail leur contenu:

In [ ]:
objectColumns = list(data.dtypes[data.dtypes == np.object].index)
numericColumns = list(data.dtypes[data.dtypes != np.object].index)
print(objectColumns)
print(numericColumns)

In [ ]:
for column in objectColumns:
    print('{}: {} uniques values'.format(column,len(data[column].unique())))

Certaines colonnes sont liées à l'identification du batiment. C'est le cas de 'PropertyName', 'Address'. Nous allons donc les enlever car peu exploitable. De plus si on voulait utiliser une technique liée à la proximité entre les batiments, la longitude et la latitude serait plus facilement utilisable que l'adresse. Les autres sont des variables catégorielles. 

In [ ]:
data = data.drop(['PropertyName', 'Address'], axis=1)

Idem pour 'OSEBuildingID', 'TaxParcelIdentificationNumber' dans les variables numériques.

In [ ]:
data = data.drop(['OSEBuildingID', 'TaxParcelIdentificationNumber'], axis=1)

In [ ]:
# suite à la suppression des outliers et à nos nouvelles colonnes par surface on peut supprimer  les anciennes colonnes
# data.drop(['SiteEnergyUse(kBtu)', 'TotalGHGEmissions'], axis=1, inplace=True)

In [ ]:
# on remet à jour la liste des colonnes catégorielles
objectColumns = list(data.dtypes[data.dtypes == np.object].index)
numericColumns = list(data.dtypes[data.dtypes != np.object].index)
print(objectColumns)
print(numericColumns)

N'ayant pas fait d'analyse poussée sur les outliers je vais utiliser le RobustScaler (les statistiques de centrage et de mise à l'échelle de RobustScaler sont basées sur des centiles et ne sont donc pas influencées par un petit nombre de valeurs aberrantes marginales très importantes) pour les valeurs numériques et le OneHotEncoderpour les catégories

In [ ]:
#columns_to_drop=['SiteEnergyUse(kBtu)','Energy/Surface', 'TotalGHGEmissions', 'GHG/Surface']
#y_columns=['Energy/Surface', 'GHG/Surface']
y_columns = ['TotalGHGEmissions', 'SiteEnergyUse(kBtu)']
X = data.drop(y_columns, axis=1)

print(X.shape)
y = data[y_columns]
print(y.shape)
print(len(numericColumns))
for i in y_columns:
    numericColumns.remove(i)
print(len(numericColumns))

# X = data.drop(['TotalGHGEmissions', 'SiteEnergyUse(kBtu)'], axis=1)
# y = data(['TotalGHGEmissions'])

In [ ]:
# standardiser les données
preprocessor = make_column_transformer((RobustScaler(),numericColumns),(OneHotEncoder(handle_unknown = 'ignore'),objectColumns))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

##  <a id="outliers">Cas des outliers<a/>   
    
    J'ai fait des tests avec le jeu de données épurés des outliers mais les résultat étaient moins bon, j'ai donc décidé de continuer à travailler avec le jeu de données initiales.

# <a id="simple-linear-regression">Régression linéaire basique<a/>

In [ ]:
print(X.columns)
print(y.columns)

In [ ]:
model = make_pipeline(preprocessor,LinearRegression())
model.fit(X_train,y_train)
print("score d'entrainement = ",model.score(X_train,y_train))
y_pred = model.predict(X_test)
print("score de la prédiction:")#, accuracy_score(y_test, y_pred)), 
print("MAE = ",mean_absolute_error(y_test,y_pred))
print("RMSE = ",np.sqrt(mean_squared_error(y_test,y_pred)))
print("median abs err = ",median_absolute_error(y_test,y_pred))

Le score de prédiction est assez correct, peut-il être meilleur en utilisant un modèle par variable à prédire:

In [ ]:
for column in y_columns:
    X_train, X_test, y_train, y_test = train_test_split(X, y[column], test_size=0.2)
    model = make_pipeline(preprocessor,LinearRegression())
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    print('Méthode: LinearRegression OneHotEncoder RobustScaler')
    print('+ utilisation de pipelines')
    print('Prédiction de ',column)
    print('score d\'entrainement = ',model.score(X_train,y_train))
    print("score de la prédiction: ",  model.score(X_test, y_test)), 
    print("MAE = ",mean_absolute_error(y_test,y_pred))
    print("RMSE = ",np.sqrt(mean_squared_error(y_test,y_pred)))
    print("median abs err = ",median_absolute_error(y_test,y_pred))
    print('')
 

On a fait certaines approximations avec ce modèle: certraines catégories  (celles du jeu de test) n'ont malheureusement pas été encodées. Cela générait des erreurs qu'on a décidé d'ignorer avec le paramètre handle_unknown = 'ignore' du OneHotEncoder. Il faudrait donc pour une telle  méthode se passer de l'utilisation des pîpeline ou il faudrait appliquer la modification avant au rique de créer une fuite des données?

In [ ]:
print(X.columns)
print(y.columns)

In [ ]:

X = data.drop(y_columns, axis=1)
y = data[y_columns]

# essais peu concluant avec les transformation appliquées avant.
encoder=LabelEncoder()
for column in objectColumns:
    X[column] = encoder.fit_transform(X[column])
    
encoder=StandardScaler()
X_std = encoder.fit_transform(X)

for column in y_columns:
    X_train, X_test, y_train, y_test = train_test_split(X_std, y[column], test_size=0.2)
    model = LinearRegression()
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    print('Méthode: LinearRegression LabelEncoder StandardScaler')
    print('Prédiction de ',column)
    print('score d\'entrainement = ',model.score(X_train,y_train))
    print("score de la prédiction:"), 
    print("MAE = ",mean_absolute_error(y_test,y_pred))
    print("RMSE = ",np.sqrt(mean_squared_error(y_test,y_pred)))
    print("median abs err = ",median_absolute_error(y_test,y_pred))
    print('')

In [ ]:
# essais peu concluant avec les transformation appliquées avant.
encoder= LabelBinarizer()
for column in objectColumns:
    X[column] = encoder.fit_transform(X[column])
    
encoder=StandardScaler()
X_std = encoder.fit_transform(X)

for column in y_columns:
    X_train, X_test, y_train, y_test = train_test_split(X_std, y[column], test_size=0.2)
    model = LinearRegression()
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    print('Méthode: LinearRegression LabelBinarizer StandardScaler')
    print('Prédiction de ',column)
    print('score d\'entrainement = ',model.score(X_train,y_train))
    print("score de la prédiction:"), 
    print("MAE = ",mean_absolute_error(y_test,y_pred))
    print("RMSE = ",np.sqrt(mean_squared_error(y_test,y_pred)))
    print("median abs err = ",median_absolute_error(y_test,y_pred))
    print('')

**Conclusion**

Le choix des transformers utilisés pour standardiser nos données est très impactant sur nos résultat. Un LabelEncoder ou un  LabelBinarizer associés avec des StandardScaler donne des résultats très médiocres. Un pipeline utilisant RobustScaler et OneHotEncoder donne des résultats bien meilleur et totalement admissible mais obligent par contre à ignorer certaines lignes dont les catégories se retrouvent dans le jeu d'entrainement mais pas dans le jeu de test et ne sont par conséquent pas connues du modèle. On essaiera de gommer ces imperfections dans la suite avec une validation croisée.

essai raté avec le StratifiedShuffleSplit...

In [ ]:
n_bins = 20
fig = plt.figure(figsize=(18,9))
plt.hist(data['TotalGHGEmissions'],n_bins)
plt.title('TotalGHGEmissions')
plt.show()
fig = plt.figure(figsize=(18,9))
plt.hist(np.log(data['TotalGHGEmissions']),n_bins)
plt.title('log TotalGHGEmissions')
plt.show()

On pourrait justifier un passage au log pour améliorer la précision des algorithme mais le RobustScaler nous permet de nous affranchir de cette transormation car il prend déjà en charge les changements d'échelles et nivelle les différences importantes et dans la pratique nous a donné de meilleurs résultats.

# <a id="model-compare">Comparaison des modèles<a/> 

In [ ]:
results = []
algos = {
    'LinearRegression' : LinearRegression(),
    'Ridge' : Ridge(),
    'Lasso' : Lasso(tol=0.5),
    'ElasticNet' : ElasticNet(),
    'SGDRegressor': SGDRegressor(),
    'SVR': SVR(),
    'RandomForestRegressor' : RandomForestRegressor(),
    'XGBRegressor' : XGBRegressor()
}
X_train, X_test, y_train_all, y_test_all = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
for algo_name, algo in algos.items():
    print('Algorithme: ',algo_name)
    for column in y_columns:
        y_test = y_test_all[column]
        y_train = y_train_all[column]
        model = make_pipeline(preprocessor,algo)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        print('Prédiction de ',column)
        print('score d\'entrainement = ',model.score(X_train,y_train))
        print("score de la prédiction: ",  model.score(X_test, y_test))
        mae = mean_absolute_error(y_test,y_pred)
        rmse = np.sqrt(mean_squared_error(y_test,y_pred))
        med_abs_err = median_absolute_error(y_test,y_pred)
        print("MAE = ", mae)        
        print("RMSE = ",rmse)
        print("median abs err = ", med_abs_err)
        print('')
        results.append([algo_name, column, model.score(X_test, y_test), mae, rmse, med_abs_err])
    print('-'*100)

# <a id="model-compare-without-ESS">Comparaison des modèles sans le EnergyStarScore<a/> 

In [ ]:
X_train = X_train.drop(['ENERGYSTARScore'], axis=1)
X_test = X_test.drop(['ENERGYSTARScore'], axis=1)
results_without_energyStarScore = []
numericColumns.remove('ENERGYSTARScore')
print(numericColumns)
preprocessor = make_column_transformer((RobustScaler(),numericColumns),(OneHotEncoder(handle_unknown = 'ignore'),objectColumns))

In [ ]:
for algo_name, algo in algos.items():
    print('Algorithme: ',algo_name)
    for column in y_columns:
        y_test = y_test_all[column]
        y_train = y_train_all[column]
        model = make_pipeline(preprocessor,algo)
        model.fit(X_train,y_train)
        y_pred = model.predict(X_test)
        print('Prédiction de ',column)
        print('score d\'entrainement = ',model.score(X_train,y_train))
        print("score de la prédiction: ",  model.score(X_test, y_test))
        mae = mean_absolute_error(y_test,y_pred)
        rmse = np.sqrt(mean_squared_error(y_test,y_pred))
        med_abs_err = median_absolute_error(y_test,y_pred)
        print("MAE = ", mae)        
        print("RMSE = ",rmse)
        print("median abs err = ", med_abs_err)
        print('')
        results_without_energyStarScore.append([algo_name, column, model.score(X_test, y_test), mae, rmse, med_abs_err])
    print('-'*100)

# <a id="conclude">Conclusion<a/>

In [ ]:
df_results = pd.DataFrame(results,columns=['algorithm', 'column','predict score', 'MAE', 'RMSE', 'median abs err'])
display(df_results.sort_values(by=['column','predict score'],ascending=False))
print("Sans le EnergyStarScore:")
df_results_without_energyStarScore = pd.DataFrame(results_without_energyStarScore,columns=['algorithm', 'column','predict score', 'MAE', 'RMSE', 'median abs err'])
display(df_results_without_energyStarScore.sort_values(by=['column','predict score'],ascending=False))

    Le XGBRegresson et le RandomForestRegressor sont nos deux algorithmes les plus performants. Ils obtiennent des résultats très satisfaisants. Cependant une optimisation des paramètres des différents algorithmes pourrait créer des différences. On va donc chercher à optimiser les paramètres de ces différents algorithme par le biais d'une validation croisée. Nous supprimerons néanmoins le SGDRegressor qui est totalement contre-performant et le SVR qui a des résultats pas assez bon.